In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
# 读取数据
marketing = pd.read_csv(r"C:\Users\leo\Desktop\QBUS6600\Data\Project1Data.csv")

postcode = pd.read_csv(r"C:\Users\leo\Desktop\QBUS6600\Data\PostCodeData.csv")
# 将postcode的DOMINANT_MOSAIC_GROUP，DOMINANT_MOSAIC_TYPE join到marketing上，连接列为PostCode（marketing）和POSTCODE
postcode_subset = postcode[['POSTCODE', 'DOMINANT_MOSAIC_GROUP', 'DOMINANT_MOSAIC_TYPE']].copy()
# 确保PostCode和POSTCODE为字符串，便于join
marketing['PostCode'] = marketing['PostCode'].astype(str)
postcode_subset['POSTCODE'] = postcode_subset['POSTCODE'].astype(str)
# 合并
marketing = marketing.merge(postcode_subset, left_on='PostCode', right_on='POSTCODE', how='left')

#Mosaic = pd.read_excel(r"C:\Users\leo\Desktop\QBUS6600\Data\Mosaic.xlsx", sheet_name='Rankings')

In [5]:
marketing = marketing[marketing['Initial_90_Days'] == 1]

# 填充marketing中的缺失值为'Unknown'
marketing = marketing.fillna('Unknown')
# 单独将IsEmergencyGift列的缺失值填充为'None'
marketing['IsEmergencyGift'] = marketing['IsEmergencyGift'].replace('Unknown', np.nan)  # 先还原为NaN，防止前面已被填充
marketing['IsEmergencyGift'] = marketing['IsEmergencyGift'].fillna('None')

# 对Next_24_Month_Value_LTV进行对数变换，添加新列'Log_Next_24_Month_Value_LTV'
marketing['Log_Next_24_Month_Value_LTV'] = np.log1p(marketing['Next_24_Month_Value_LTV'])

# 1) 预处理：一次性转为 datetime（cache=True 更快）
marketing['GiftDate'] = pd.to_datetime(marketing['GiftDate'], errors='coerce', cache=True)

# 根据GiftDate生成季节列，1=春，2=夏，3=秋，4=冬（南半球季节）
def get_season(dt):
    if pd.isnull(dt):
        return np.nan
    month = dt.month
    if month in [9, 10, 11]:
        return 1  # 春
    elif month in [12, 1, 2]:
        return 2  # 夏
    elif month in [3, 4, 5]:
        return 3  # 秋
    elif month in [6, 7, 8]:
        return 4  # 冬
    else:
        return np.nan

marketing['Gift_Season'] = marketing['GiftDate'].apply(get_season)

In [ ]:
marketing = marketing.copy()

# ---- State 清洗与分箱 ----
AU_STATES = {"NSW", "VIC", "QLD", "SA", "WA", "TAS", "NT", "ACT"}
INT_TOKENS = {"INT", "INTERNATIONAL", "OVERSEAS", "OS", "INTL"}

# 新西兰常见地区代码（含你数据里出现的 & 常见三位缩写）
NZ_CODE_SET = {
    "AUK", "WGN", "CAN", "OTA", "WKO", "WYK", "BOP", "HKB", "TKI", "MWT", "GIS", "NSN", "TAS", "MBH", "WTC", "STL", "NTL"
}
NZ_NAME_SET = {
    "AUCKLAND", "WELLINGTON", "CANTERBURY", "OTAGO", "WAIKATO", "BAYOFPLENTY", "HAWKESBAY", "GISBORNE",
    "MANAWATUWHANGANUI", "MANAWATUWANGANUI", "NELSON", "TASMAN", "MARLBOROUGH", "WESTCOAST", "SOUTHLAND", "NORTHLAND"
}

def classify_state(x):
    if pd.isna(x):
        return np.nan

    s = str(x).strip().upper()

    # 规范化：仅字母、仅数字
    letters_only = re.sub(r'[^A-Z]', '', s)
    digits_only  = re.sub(r'[^0-9]', '', s)

    # 1) 澳洲州别（优先匹配，避免 "TAS" 与 NZ Tasman 冲突）
    if letters_only in AU_STATES:
        return f"{letters_only}"

    # 2) 明确表示海外
    if letters_only in INT_TOKENS:
        return "OVERSEAS"

    # 3) 新西兰（只要是新西兰地区代码或名称都归为"NZ"）
    if letters_only in NZ_CODE_SET or letters_only in NZ_NAME_SET:
        return "New Zealand"

    # 4) 无意义字符（仅数字；或去标点后为空；或只有1个字母）
    if letters_only == "" or len(letters_only) == 1 or (digits_only != "" and letters_only == ""):
        return "NONSENSE"

    # 5) 其他一律视为海外（如国家/城市/州缩写等）
    return "OVERSEAS"

marketing["State_upper"]  = marketing["State"].astype(str).str.strip().str.upper()
marketing.loc[marketing["State"].isna(), "State_upper"] = np.nan
marketing["State_bucket"] = marketing["State"].apply(classify_state)

marketing = marketing.drop(columns=['POSTCODE', 'State', 'State_upper'])

**关于变量聚合，还需讨论后再填充**

In [ ]:
train_idx, test_idx = train_test_split(
    marketing.index,
    test_size=0.2,
    random_state=42
)

marketing_train = marketing.loc[train_idx].reset_index(drop=True)
marketing_test = marketing.loc[test_idx].reset_index(drop=True)

**下面的部分用不上，不需要看。**

In [ ]:
# 对df按SupporterID去重，只保留一条记录，并保留指定列（去除所有含有score和rank的列）
keep_cols = [
    'SupporterID', 'Next_24_Month_Value_LTV',
    'DOMINANT_MOSAIC_TYPE', 'DOMINANT_MOSAIC_GROUP',
    'State_bucket', 'Age_Bucket', 'Gender', 'PostCode'
]
df_filtered_copy = marketing[keep_cols].drop_duplicates(subset=['SupporterID'])

In [ ]:
# 2) 全局最大日期
max_giftdate = marketing['GiftDate'].max()

# 3) 更快的众数函数（value_counts 比 mode 快；无值返回 NaN）
def fast_mode(s: pd.Series):
    vc = s.value_counts(dropna=True)
    return vc.index[0] if len(vc) else np.nan

# 4) 用 idxmax 找到每个 SupporterID 的“最近一条”记录（避免排序+first）
idx_last = marketing.groupby('SupporterID', observed=True)['GiftDate'].idxmax()
recent = (
    marketing.loc[idx_last, ['SupporterID', 'GiftDate', 'GiftAmount']]
      .rename(columns={'GiftAmount': 'Recent_GiftAmount'})
      .set_index('SupporterID')
)
recent['Days_Since_Last_Gift'] = (max_giftdate - recent['GiftDate']).dt.days
recent = recent[['Days_Since_Last_Gift', 'Recent_GiftAmount']]

In [ ]:
# 对每个SupporterID判断是否有IsEmergencyGift为'Yes'的记录
emergency_flag = (
    marketing.groupby('SupporterID')['IsEmergencyGift']
      .apply(lambda x: 'Yes' if (x == 'Yes').any() else 'No')
      .reset_index(name='Has_EmergencyGift')
)

# 获取每个SupporterID对应的Next_24_Month_Value_LTV的第一个值
ltv_first = (
    marketing.groupby('SupporterID')['Next_24_Month_Value_LTV']
      .first()
      .reset_index()
)

# 合并两个结果，只保留SupporterID、Has_EmergencyGift和Next_24_Month_Value_LTV
df_emergency_ltv = pd.merge(emergency_flag, ltv_first, on='SupporterID')


In [ ]:
import numpy as np
import pandas as pd

# 1) 预处理：一次性转为 datetime（cache=True 更快）
marketing['GiftDate'] = pd.to_datetime(marketing['GiftDate'], errors='coerce', cache=True)

# 2) 全局最大日期
max_giftdate = marketing['GiftDate'].max()

# 3) 更快的众数函数（value_counts 比 mode 快；无值返回 NaN）
def fast_mode(s: pd.Series):
    vc = s.value_counts(dropna=True)
    return vc.index[0] if len(vc) else np.nan

# 4) 用 idxmax 找到每个 SupporterID 的“最近一条”记录（避免排序+first）
idx_last = marketing.groupby('SupporterID', observed=True)['GiftDate'].idxmax()
recent = (
    marketing.loc[idx_last, ['SupporterID', 'GiftDate', 'GiftAmount']]
      .rename(columns={'GiftAmount': 'Recent_GiftAmount'})
      .set_index('SupporterID')
)
recent['Days_Since_Last_Gift'] = (max_giftdate - recent['GiftDate']).dt.days
recent = recent[['Days_Since_Last_Gift', 'Recent_GiftAmount']]

# 5) 聚合统计：一次 groupby 完成 count/mean/var 及“众数”字段
agg_all = (
    marketing.groupby('SupporterID', observed=True).agg(
        # GiftAmount_count=('GiftAmount', 'size'),
        # GiftAmount_mean =('GiftAmount', 'mean'),
        # GiftAmount_var  =('GiftAmount', 'var'),
        Next_24_Month_Value_LTV=('Next_24_Month_Value_LTV', 'first'),
        Have_Phone = ('Have_Phone','first'),
        Have_Email = ('Have_Email','first'),
        # 以下暂时不需要分析的部分注释掉，避免语法错误
        # AppealSeason_mode=('AppealSeason', fast_mode),
        # CampaignSubtype_Group_mode=('CampaignSubtype_Group', fast_mode),
        GiftSolicitationChannel_mode=('GiftSolicitationChannel', fast_mode),
        # ProductType_Group_mode=('ProductType_Group', fast_mode),
    )
)

# 6) 直接用 join（索引对齐，比 merge+reset_index 更省）
df_supporter = agg_all.join(recent, how='left').reset_index()
